In [2]:
from bs4 import BeautifulSoup as bs
import requests
from fake_useragent import UserAgent
import json
import pandas as pd

In [3]:
ua = UserAgent(verify_ssl=False)
fake_ua = ua.random
headers = {
    'user-agent' : fake_ua
}

In [4]:
# 상품명 product_name
# 상품 이미지 URL 리스트 image_list
# 가격 price
# 상품 정보 info
# 분류 1 cat_1...
# 게시일 date

In [34]:
# 카테고리id, 카테고리명 가져오기
# categories = { id : 카테고리명 }
# 310 대분류 - 310090 중분류 - 310090050 소분류 로 확장됨

def crawl_categories(categories_list):
    cat_dict = {}
    for categories in categories_list:
        category_id = categories['id']
        category_title = categories['title']
        cat_dict[category_id] = category_title
        
        sub_categories = categories.get('categories')
        if sub_categories:
            cat_dict.update(crawl_categories(sub_categories))

    return cat_dict 

CATLIST_URL = 'https://api.bunjang.co.kr/api/1/categories/list.json'
categories_list = json.loads(requests.get(CATLIST_URL).text)['categories']

cat_id_dict = crawl_categories(categories_list)
cat_id_dict

{'310': '여성의류',
 '310090': '패딩/점퍼',
 '310090050': '롱패딩',
 '310090060': '숏패딩',
 '310090020': '블루종/항공점퍼',
 '310090999': '기타(패딩/점퍼)',
 '310110': '코트',
 '310110030': '겨울 코트',
 '310110010': '봄/가을 코트',
 '310240': '맨투맨',
 '310230': '후드티/후드집업',
 '310260': '티셔츠',
 '310260200': '긴팔 티셔츠',
 '310260100': '반팔 티셔츠',
 '310260300': '민소매 티셔츠',
 '310040': '블라우스',
 '310040090': '긴팔 블라우스',
 '310040080': '반팔 블라우스',
 '310040070': '민소매 블라우스',
 '310040999': '기타(블라우스)',
 '310050': '셔츠',
 '310050010': '솔리드(단색) 셔츠',
 '310050050': '스트라이프 셔츠',
 '310050030': '체크 셔츠',
 '310050040': '청/데님 셔츠',
 '310050999': '기타(셔츠)',
 '310150': '바지',
 '310150030': '면바지',
 '310150010': '슬랙스',
 '310150040': '조거팬츠',
 '310150070': '레깅스',
 '310150999': '기타(바지)',
 '310140': '청바지',
 '310140020': '일자 청바지',
 '310140010': '스키니진',
 '310140030': '부츠컷 청바지',
 '310140040': '배기 청바지',
 '310140999': '기타(청바지)',
 '310160': '반바지',
 '310160010': '면 반바지',
 '310160060': '밴딩 반바지',
 '310160020': '데님 반바지',
 '310160999': '기타(반바지)',
 '310130': '치마',
 '310130030':

In [5]:
### 카테고리id가 대-중-소분류 구분없이 한 딕셔너리에 들어가있어 카테고리별로 다 크롤링하면 제품이 중복됨
### 1. 딕셔너리 형태를 바꾸기 - 뇌절(근데 나라가 이렇게 한듯)
### 2. 카테고리별로 다 돌리되 중복되면 append하지 않도록 - 비효율적? 일단 이렇게 함 



# 반복문으로 각 카테고리 접근
for cat_id in cat_id_dict:
    CAT_URL = 'https://api.bunjang.co.kr/api/1/find_v2.json?f_category_id={}&page=0&order=date&req_ref=popular_category&request_id=2023517000440&stat_device=w&n=100&version=4'.format(cat_id)

    # 해당 카테고리의 제품id(pid) 가져오기 
    json_product = json.loads(requests.get(CAT_URL).text)

    pid_list = []
    for i in range(len(json_product['list'])):
        pid = json_product['list'][i]['pid']
        if pid not in pid_list:
            pid_list.append(pid)

In [ ]:
# CAT_URL 로 제품id 가져오기
# 가져온 제품id로 PRODUCT_URL들어가 제품명, 가격, 제품이미지url, 제품설명, 게시일(여기서 가져올지 위에서 가져와서 datetime쓸지 고민) 가져오기

In [28]:
# 제품 상세페이지 url 크롤링
for id in pid_list:
    product_id = id
    PRODUCT_URL = 'https://api.bunjang.co.kr/api/pms/v1/products-detail/{}?viewerUid=-1'.format(product_id)
    json_product = json.loads(requests.get(PRODUCT_URL).text)

In [30]:
product_name = json_product['data']['product']['name'] # 제품명
image_url = json_product['data']['product']['imageUrl'] # 이미지url
price = json_product['data']['product']['price'] # 가격
info = json_product['data']['product']['description'].replace('\n', ' ').strip() # 제품설명
date = json_product['data']['product']['updatedAt'] # date 형태가 '2023-05-16T16:18:23.422Z'

In [31]:
product_name, image_url, price, info, date

('새상품/가격인하) 유니크 하프 벨트장식 테니스 면스커트S',
 'https://media.bunjang.co.kr/product/185253076_{cnt}_1679571579_w{res}.jpg',
 14000,
 '시착만 해본 새상품 스커트입니다!벨트 장식 포인트 있구요!반반 느낌이라 유니크라서 예뻐요😊면소재 두툼하고 빳빳한 원단입니다😊핏은 예쁜데 저한테 좀 커서 내놓아요..!!',
 '2023-05-16T16:18:23.422Z')

In [36]:
# CAT_URL에서 게시일 가져올 시 

import datetime

timestamp = 1684198699
dt = datetime.datetime.fromtimestamp(timestamp)
formatted_time = dt.strftime("%Y-%m-%d %H:%M:%S")

print(formatted_time)


2023-05-16 09:58:19
